In [1]:
"""
Fine-grained classification practice with Flower-17
"""

# Python Packages
import argparse
import os
import time
# 3rd Party Packages
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
import cv2
# User Packages
from start.preprocessing import ImageToTensorPreprocessor, ResizePreprocessor, ColorSpacePreprocessor
from start.loader import ImageCachedDataset
from start.model import MiniVGGNet


In [2]:
# Load Flowers-17 dataset
dataset = ImageCachedDataset(
    preprocessors=[
        ResizePreprocessor(224, 224, aspect_preserving=True),
        ColorSpacePreprocessor(conversion=cv2.COLOR_BGR2GRAY),
        ImageToTensorPreprocessor()
    ],
    dataset_path=r'T:\temp\simeon\dataset\custom'
)
dataset._ignored_labels.append('firearm')
"""
(data, labels) = dataset.load(
    dataset_path=r'/home/share/dataset/flowers17',
    verbosity=80
)
"""
(data, labels) = dataset.load(
    verbosity=80
)

classes = set(labels)

print('data shape: {}'.format(data.shape))
print('labels shape: {}'.format(labels.shape))
print('classes: {}'.format(classes))


# Normalize data
data = data.astype(np.float) / 255.0

# Split into train and test sets
(trainX, testX, trainY, testY) = train_test_split(
    data, labels,
    test_size=0.2,
    random_state=int(time.time()),
    stratify=list(labels)
)

# Free up the memory
del data
del labels

[INFO] Processing label: person
[INFO] processed 80 person images
[INFO] processed 160 person images
[INFO] processed 240 person images
[INFO] processed 320 person images
[INFO] processed 400 person images
[INFO] processed 480 person images
[INFO] processed 560 person images
[INFO] processed 640 person images
[INFO] processed 720 person images
[INFO] processed 800 person images
[INFO] processed 880 person images
[INFO] processed 960 person images
[INFO] processed 1040 person images
[INFO] Processing label: soldier
[INFO] processed 80 soldier images
[INFO] processed 160 soldier images
[INFO] processed 240 soldier images
[INFO] processed 320 soldier images
[INFO] processed 400 soldier images
[INFO] processed 480 soldier images
[INFO] processed 560 soldier images
[INFO] processed 640 soldier images
[INFO] processed 720 soldier images
[INFO] processed 800 soldier images
[INFO] processed 880 soldier images


TypeError: 'NoneType' object is not iterable

In [ ]:
# Setup data splits
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import TensorBoard
import pycm

N_EPOCHS = 35
BATCH_SIZE = 32
n_classes = len(classes)
kfold_splits = 10 #empirical
timestamp = time.time()

# Instantiate the cross validator
skf = StratifiedKFold(n_splits=kfold_splits, shuffle=True)

# Convert output to either binarized one-hot vectors for categorical or 0/1 for binary
if n_classes > 2:
    lb = LabelBinarizer()
    testY = lb.fit_transform(testY)
else:
    le = LabelEncoder()
    testY = le.fit_transform(testY)
    
# Data augmentation
augmenter = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

def create_model(n_classes, alpha=1.4):
    # Initialize the optimizer and model
    print('[INFO] compiling model...')
    opt = SGD(lr=0.05)
    properties = {
        'width':    64,
        'height':   64,
        'channels': 3,
        'classes':  len(classes)
    }
    #model = MiniVGGNet.build(properties)
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense
    model = Sequential()
    model.add(MobileNetV2(
        input_shape=(224, 224, 3),
        alpha=alpha, 
        depth_multiplier=1, 
        include_top=False, 
        weights='imagenet', 
        input_tensor=None, 
        pooling='avg'
    ))

    if n_classes > 2:
        model.add(Dense(
            units=n_classes,
            activation='softmax',
            use_bias=True,
            kernel_initializer='glorot_uniform',
            bias_initializer='glorot_uniform',
            kernel_regularizer=None,
            bias_regularizer=None,
            activity_regularizer=None,
            kernel_constraint=None,
            bias_constraint=None
        ))
        model.compile(
            loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy']
        )
    # Hot dog or not hot dog
    else:
        model.add(Dense(
            units=1,
            activation='sigmoid',
            use_bias=True,
            kernel_initializer='glorot_uniform',
            bias_initializer='glorot_uniform',
            kernel_regularizer=None,
            bias_regularizer=None,
            activity_regularizer=None,
            kernel_constraint=None,
            bias_constraint=None
        ))
        model.compile(
            loss='binary_crossentropy',
            optimizer=opt,
            metrics=['accuracy']
        )

    return model

# Create model
#model = create_model(n_classes=n_classes)
from start.model import MobileNetV2
model = MobileNetV2.build({
    'width':    224,
    'height':   224,
    'channels': 1,
    'weights': 'imagenet',
    'dense_units': 242,
    'dropout_rate': 0.43,
    'regularization_strength': 0.0001
})

for index, (train_indices, val_indices) in enumerate(skf.split(trainX, trainY)):
    print('Training on fold {}/{}'.format(index, kfold_splits))
    # Partition into train and test splits
    """
    (valX, testX, valY, testY) = train_test_split(
        testX, testY,
        test_size=0.4,
        random_state=int(time.time()),
        stratify=list(testY)
    )
    """
    trainSplitX, valSplitX = trainX[train_indices], trainX[val_indices]
    trainSplitY, valSplitY = trainY[train_indices], trainY[val_indices]
    
    # Convert output to either binarized one-hot vectors for categorical or 0/1 for binary
    if n_classes > 2:
        trainSplitY = lb.fit_transform(trainSplitY)
        valSplitY = lb.fit_transform(valSplitY)
    else:
        trainSplitY = le.fit_transform(trainSplitY)
        valSplitY = le.fit_transform(valSplitY)

    # Initialize TensorBoard
    """
    tb_callback = TensorBoard(
        log_dir='./logs/splitindex{}'.format(index), 
        histogram_freq=1, 
        batch_size=BATCH_SIZE, 
        write_graph=False, 
        write_grads=False, 
        write_images=False, 
        embeddings_freq=0,
        embeddings_layer_names=None, 
        embeddings_metadata=None, 
        embeddings_data=None
    )
    """


    # Train the network
    print('[INFO] training network split {}...'.format(index))
    callbacks = []
    try:
        if tb_callback is not None:
            callbacks.append(tb_callback)
    except NameError:
        pass
    history = model.fit_generator(
        augmenter.flow(trainSplitX, trainSplitY, 
                       batch_size=BATCH_SIZE),
        validation_data=(valSplitX, valSplitY),
        steps_per_epoch=len(trainX) // BATCH_SIZE,
        epochs=N_EPOCHS,
        callbacks=callbacks,
        verbose=1
    )
            
    # Evaluate the network
    print('[INFO] evaluating network split {}...'.format(index))
    predictions = model.predict(testX, batch_size=BATCH_SIZE)
    if not n_classes > 2:
        threshold = 0.5
        predictions_probability = predictions
        predictions[predictions>threshold] = 1
        predictions[predictions<=threshold] = 0
        predictions = predictions.astype(np.int)
        

    if n_classes > 2:
        cm = pycm.ConfusionMatrix(
            actual_vector=lb.inverse_transform(testY),
            predict_vector=lb.inverse_transform(predictions)
        )
    else:
        cm = pycm.ConfusionMatrix(
            actual_vector=le.inverse_transform(testY),
            predict_vector=le.inverse_transform(predictions)
        )
    cm.save_html(r'T:\temp\simeon\dataset\{}_confusion_matrix_splitindex{}'.format(timestamp, index))

    # Plot the training loss and accuracy
    plt.style.use('ggplot')
    plt.figure()
    plt.plot(np.arange(0, N_EPOCHS), history.history['loss'], label='train_loss')
    plt.plot(np.arange(0, N_EPOCHS), history.history['val_loss'], label='val_loss')
    plt.plot(np.arange(0, N_EPOCHS), history.history['acc'], label='train_acc')
    plt.plot(np.arange(0, N_EPOCHS), history.history['val_acc'], label='val_acc')
    plt.title('Training Loss and Accuracy')
    plt.xlabel('Epoch #')
    plt.ylabel('Loss/Accuracy')
    plt.legend()
    plt.savefig(r'T:\temp\simeon\dataset\{}_training_splitindex{}.jpg'.format(timestamp, index))

In [ ]:
# Evaluate the network
predictions = model.predict(testX, batch_size=BATCH_SIZE)
if not n_classes > 2:
    threshold = 0.5
    predictions_probability = predictions.copy()
    predictions[predictions>threshold] = 1
    predictions[predictions<=threshold] = 0
    predictions = predictions.astype(np.int)

# Plot images that were soldiers classified as people
print('classes: {}'.format(classes))
print('testY shape: {}'.format(testY.shape))
print('predictions shape: {}'.format(predictions.shape))
soldiers_classified_people = testX[np.logical_and(testY == 1, np.squeeze(predictions) == 0)]
try:
    probability = predictions_probability[np.logical_and(testY == 1, np.squeeze(predictions) == 0)]
except NameError:
    pass
print('soldiers_classified_people shape: {}'.format(soldiers_classified_people.shape))
print('probability shape: {}'.format(probability.shape))
print('probability: \n{}'.format(probability))
# Draw figure
if soldiers_classified_people.shape[0] > 0:
    IMAGES_PER_ROW = 4
    rows = (soldiers_classified_people.shape[0] // IMAGES_PER_ROW) + 1
    f = plt.figure(
        num=1, 
        figsize=(14, int(14*rows/float(IMAGES_PER_ROW))), 
        dpi=80, 
        facecolor='w', 
        edgecolor='k'
    )
    print('{} x {} plot'.format(rows, IMAGES_PER_ROW))
    for subidx in range(soldiers_classified_people.shape[0]):
        # subplot is 1-indexed
        plt.subplot(rows, IMAGES_PER_ROW, subidx+1)
        f.gca().grid(False)
        try:
            plt.title('Soldier: {0:.2f}%'.format(np.asscalar(probability[subidx])*100.0))
        except NameError:
            pass
        plt.imshow(soldiers_classified_people[subidx][..., ::-1])
    plt.show()

In [ ]:
# Evaluate the network
predictions = model.predict(testX, batch_size=BATCH_SIZE)
if not n_classes > 2:
    threshold = 0.5
    predictions_probability = predictions.copy()
    predictions[predictions>threshold] = 1
    predictions[predictions<=threshold] = 0
    predictions = predictions.astype(np.int)

# Plot images that were people classified as soldiers
print('classes: {}'.format(classes))
print('testY shape: {}'.format(testY.shape))
print('predictions shape: {}'.format(predictions.shape))
people_classified_soldiers = testX[np.logical_and(testY == 0, np.squeeze(predictions) == 1)]
try:
    probability = predictions_probability[np.logical_and(testY == 0, np.squeeze(predictions) == 1)]
except NameError:
    pass
print('people_classified_soldiers shape: {}'.format(people_classified_soldiers.shape))
print('probability shape: {}'.format(probability.shape))
print('probability: \n{}'.format(probability))
# Draw figure
if people_classified_soldiers.shape[0] > 0:
    IMAGES_PER_ROW = 4
    rows = (people_classified_soldiers.shape[0] // IMAGES_PER_ROW) + 1
    f = plt.figure(
        num=2, 
        figsize=(14, int(14*rows/float(IMAGES_PER_ROW))), 
        dpi=80, 
        facecolor='w', 
        edgecolor='k'
    )
    print('{} x {} plot'.format(rows, IMAGES_PER_ROW))
    for subidx in range(people_classified_soldiers.shape[0]):
        # subplot is 1-indexed
        plt.subplot(rows, IMAGES_PER_ROW, subidx+1)
        f.gca().grid(False)
        try:
            plt.title('Person: {0:.2f}%'.format(np.asscalar(1.0-probability[subidx])*100.0))
        except NameError:
            pass
        plt.imshow(people_classified_soldiers[subidx][..., ::-1])
    plt.show()

In [ ]:
# Save the model!
model.save_weights(r'T:\temp\simeon\dataset\{}_mnv2_soldier-person_StratifiedKFold10_weights.h5'.format(timestamp))

In [ ]:
model.summary()

In [ ]:
model.layers[0].summary()